In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import ast
from pyspark.sql.types import StringType
from pyspark import SQLContext , SparkContext
from pyspark.sql.session import SparkSession

In [5]:
sc = SparkContext('local','example')  # if using locally
sqlContext = SQLContext(sc)
spark = SparkSession(sc)

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=example, master=local) created by __init__ at <ipython-input-1-696c9701050e>:4 

In [6]:
#Employee_rdd = sc.textFile("final_all_3.1.csv").map(lambda line: line.split(","))
#Employee_df = Employee_rdd.toDF(['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','batsman','bowler','date','extras.byes','extras.legbyes','extras.noballs','extras.penalty','extras.s','extras.wides','index_all','non_striker','replacements.match','replacements.role','runs.batsman','runs.extras','runs.non_boundary','runs.total','team','wicket','wicket.fielders','wicket.kind','wicket.player_out','info.bowl_out','info.city','info.competition','info.dates','info.gender','info.match_type','info.neutral_venue','info.outcome.bowl_out','info.outcome.by.innings','info.outcome.by.runs','info.outcome.by.wickets','info.outcome.eliminator','info.outcome.method','info.outcome.result','info.outcome.winner','info.overs','info.player_of_match','info.supersubs.Australia','info.supersubs.Bangladesh','info.supersubs.India','info.supersubs.New Zealand','info.supersubs.Pakistan','info.supersubs.South Africa','info.supersubs.Sri Lanka','info.supersubs.West Indies','info.teams','info.toss.decision','info.toss.winner','info.umpires','info.venue','overs','over_no'])
#df = pd.read_csv("final_all_3.1.csv")

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (15,16,26,33,37,38,39,43,44,46,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
df = (spark.read.format("csv").options(header="true" , inferSchema = True).load("final_all_3.1.csv"))

#df = sqlContext.load(source="final_all_3.1.csv", path = "final_all_3.1.csv", header = True,inferSchema = True)


In [34]:
df.show()

pyspark.sql.dataframe.DataFrame

In [36]:
# Data Frame Manipulations

# To see datatype of columns
df.printSchema()

# Show first n observation
df.head(5)
# Or
df.show(2,truncate= True)


# To Count the number of rows in DataFrame
df.count()

# How many columns do we have in dataframes along with their names
len(df.columns), df.columns

# How to get the summary statistics (mean, standard deviance, min ,max , count) of numerical columns in a DataFrame
        
df.describe().show()
train.describe('index_all').show()

# To select column(s) from the DataFrame
df.select('index_all','batsman').show(5)

# To find the number of distinct index_all in dataframes
train.select('index_all').distinct().count()

#diff_cat_in_train_test=test.select('Product_ID').subtract(train.select('Product_ID'))
#diff_cat_in_train_test.distinct().count()# For distict count


#if I want to calculate pair wise frequency of categorical columns

#train.crosstab('Age', 'Gender').show()
#Output:
#+----------+-----+------+
#|Age_Gender|    F|     M|
#+----------+-----+------+
#|      0-17| 5083| 10019|
#|     46-50|13199| 32502|
#|     18-25|24628| 75032|
#|     36-45|27170| 82843|
#|       55+| 5083| 16421|
#|     51-55| 9894| 28607|
#|     26-35|50752|168835|
#+----------+-----+------+

# If I want to get the DataFrame which won’t have duplicate rows of given DataFrame
df.select('index_all','batsman').dropDuplicates().show()

# If I want to drop the all rows with null value
df.dropna().count()

#if I want to fill the null values in DataFrame with constant number
df.fillna(-1).show(2)

# If I want to filter the rows in df which has index_all more than 1000
df.filter(df.index_all > 1000).count()

# To find the mean of each batsman group in df
df.groupby('batsman').agg({'runs': 'mean'}).show()

df.groupby('batsman').count().show()

# To create a sample DataFrame from the base DataFrame


# withReplacement = True or False to select a observation with or without replacement.
# fraction = x, where x = .5 shows that we want to have 50% data in sample DataFrame.
# seed for reproduce the result

d1 = df.sample(False, 0.2, 42)
d2 = df.sample(False, 0.2, 43)
d1.count(),d2.count()

# To apply map operation on DataFrame columns
df.select('index_all').map(lambda x:(x,1)).take(5)

# to sort the DataFrame based on column(s)
df.orderBy(df.index_all.desc()).show(5)

# to add the new column in DataFrame
df.withColumn('new_column', df.index_all /2.0).select('index_all','new_column').show(5)

# to drop a column in DataFrame
df.drop('`new.column`').columns

# if I want to remove some categories of Product_ID column in test that are not present in Product_ID column in train
#diff_cat_in_train_test=test.select('Product_ID').subtract(train.select('Product_ID'))
#diff_cat_in_train_test.distinct().count()# For distict count
#Output:
#46

# We have got 46 different categories in test. For removing these categories from the test ‘Product_ID’ column. I am applying these steps.

#Create the distinct list of categories called ‘not_found_cat’ from the diff_cat_in_train_test using map operation.
#Register a udf(user define function).
#User defined function will take each element of test column and search this in not_found_cat list and it will put -1 if  it finds in this list otherwise it will do nothing.
#Let’s see how it works. First create ‘not_found_cat’

#not_found_cat = diff_cat_in_train_test.distinct().rdd.map(lambda x: x[0]).collect()
#len(not_found_cat)
#Output: 
#46

#Now resister the udf, we need to import StringType from the pyspark.sql and udf from the pyspark.sql.functions. The udf function takes 2 parameters as arguments:

#Function (I am using lambda function)
#Return type (in my case StringType())

#from pyspark.sql.types import StringType
#from pyspark.sql.functions import udf
#F1 = udf(lambda x: '-1' if x in not_found_cat else x, StringType())

#In the above code function name is ‘F1’ and we are putting ‘-1’  
#for not found catagories in test ‘Product_ID’. Finally apply above ‘F1’ function on test 
#‘Product_ID’ and take result in k1 for new column calles “NEW_Product_ID”.
#k = test.withColumn("NEW_Product_ID",F1(test["Product_ID"])).select('NEW_Product_ID')

# Now, let’s see the results by again calculating the different categories in k and train subtract operation.
# diff_cat_in_train_test=k.select('NEW_Product_ID').subtract(train.select('Product_ID'))
# diff_cat_in_train_test.distinct().count()# For distinct count
# Output:
# 1

# The output 1 means we have now only 1 different category k and train.

#diff_cat_in_train_test.distinct().collect()
#Output:
#Row(NEW_Product_ID=u'-1')




# To Apply SQL Queries on DataFrame

df.registerAsTable('df_table')

sqlContext.sql('select index_all from df_table').show(5)

sqlContext.sql('select batsman, max(runs) from df_table group by batsman').show()



root
 |-- _c0: integer (nullable = true)
 |-- Unnamed: 0: integer (nullable = true)
 |-- Unnamed: 0.1: integer (nullable = true)
 |-- Unnamed: 0.1.1: integer (nullable = true)
 |-- batsman: string (nullable = true)
 |-- bowler: string (nullable = true)
 |-- date: string (nullable = true)
 |-- extras.byes: double (nullable = true)
 |-- extras.legbyes: double (nullable = true)
 |-- extras.noballs: double (nullable = true)
 |-- extras.penalty: double (nullable = true)
 |-- extras.s: double (nullable = true)
 |-- extras.wides: double (nullable = true)
 |-- index_all: integer (nullable = true)
 |-- non_striker: string (nullable = true)
 |-- replacements.match: string (nullable = true)
 |-- replacements.role: string (nullable = true)
 |-- runs.batsman: integer (nullable = true)
 |-- runs.extras: integer (nullable = true)
 |-- runs.non_boundary: double (nullable = true)
 |-- runs.total: integer (nullable = true)
 |-- team: string (nullable = true)
 |-- wicket: double (nullable = true)
 |-- wi